In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from datetime import datetime

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'

In [2]:
races = pd.read_csv(path+'data/races.csv')
qualifying = pd.read_csv(path+'data/qualifying.csv')
constructors = pd.read_csv(path+'data/constructor_standings.csv')
drivers = pd.read_csv(path+'data/driver_standings.csv')
results = pd.read_csv(path+'data/results.csv')
weather = pd.read_csv(path+'data/weather.csv')

In [3]:
df1 = pd.merge(races, weather, how='inner', on=['season', 'round', 'circuit_id']).drop(['lat', 'long','country','weather'], axis = 1)
df2 = pd.merge(df1, results, how='inner', on=['season', 'round', 'circuit_id']).drop(['url','points', 'status', 'time'], axis = 1)
df3 = pd.merge(df2, drivers, how='left', on=['season', 'round', 'driver']) 
df4 = pd.merge(df3, constructors, how='left', on=['season', 'round', 'constructor'])

merged_df = pd.merge(df4, qualifying, how='inner', on=['season', 'round', 'grid']).drop(['driver_y', 'constructor_y'], axis=1)

merged_df.rename(columns={'driver_x': 'driver', 'constructor_x': 'constructor'}, inplace=True)

In [4]:
# fill/drop nulls

for col in ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 
            'constructor_wins' , 'constructor_standings_pos']:
    merged_df[col].fillna(0, inplace=True)
    merged_df[col] = merged_df[col].map(lambda x: int(x))
    
merged_df.dropna(inplace = True )

In [5]:
merged_df.head()

,season,round,circuit_id,date,warm,cold,dry,wet,cloudy,driver,...,grid,podium,driver_points,driver_wins,driver_standings_pos,constructor_points,constructor_wins,constructor_standings_pos,qual_time,q_delta
0,2020,1,red_bull_ring,2020-07-05,True,False,False,False,False,bottas,...,1,1,25,1,1,37,1,1,62.939,0.000
1,2020,1,red_bull_ring,2020-07-05,True,False,False,False,False,leclerc,...,7,2,18,0,2,19,0,3,63.923,0.984
2,2020,1,red_bull_ring,2020-07-05,True,False,False,False,False,norris,...,3,3,16,0,3,26,0,2,63.626,0.687
3,2020,1,red_bull_ring,2020-07-05,True,False,False,False,False,hamilton,...,5,4,12,0,4,37,1,1,62.951,0.012
4,2020,1,red_bull_ring,2020-07-05,True,False,False,False,False,sainz,...,8,5,10,0,5,26,0,2,63.971,1.032


In [6]:
merged_df.to_csv(path+'data/merged.csv', index=False)